# MMS SITL Ground Loop: Running the GLS MP Model
This notebook runs the `mp-dl-unh` GLS Magnetopause model to make predictions. The end result is a CSV file with model selections. This notebook and the model can be run using Level-2 (L2) data, which is publicly available at the [SDC](https://lasp.colorado.edu/mms/sdc/public/), but operationally is ran using SITL- or Quick Look (QL)-level data, which is not public and requires a username and password.

<span style="font-size:larger;">**Contents**</span>
* [Front Matter](#front_matter)
* [Setting Up](#setting_up)
* [Download Data](#download_data)
  * [FGM](#download_fgm)
  * [EDP](#download_edp)
  * [DIS](#download_dis)
  * [DES](#download_des)
  * [Combine Dataframes](#combine_dataframes)
* [Run the Model](#run_model)

<a id='front_matter'></a>
## Front Matter
This section defines the packages to import, data and model directories, SDC log-in credentials, and model parameters.

In [ ]:
# Imports
import pymms
from pymms.util.tai import utc2tai
from pymms.sdc import mrmms_sdc_api as api
import pandas as pd
import numpy as np
import scipy.constants
from cdflib import cdfread, epochs
import datetime
from pathlib import Path
import pickle
import sklearn # Required to open scaler.sav file

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Disables Tensorflow debugging information
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, TimeDistributed
from tensorflow.keras.models import Sequential

The `data_root` is the directory in which all data files are downloaded. It, as well as your MMS log-in credentials, can be set more permanently in the configuration file within pymms. The `dropbox_root` is where the new model predictions generated by this notebook will be saved, and `model_root` is just the mms-sitl-ground-loop package directory where this notebook and the model parameters are kept. Here, we assume that `model_root` is in the current working directory, which [may not be true](https://github.com/ipython/ipython/issues/10123).

In [ ]:
# Log-In Credentials
#   - these can be set in pymms/config.ini
#   - you may need to set the model_root
# pymms.config['data_root'] = ''
# pymms.config['username'] = ''
# pymms.config['password'] = ''
model_root = os.getcwd()
if not os.path.isfile(os.path.join(model_root, 'model.model')):
    raise ValueError('Could not automatically determine the model root.')

# Paths
data_path_root = Path(pymms.config['data_root']).expanduser().absolute().resolve()
dropbox_root = data_path_root / 'mp_dl_unh_predictions/'
model_root = Path(model_root).expanduser().absolute().resolve()

Here, we define the spacecraft, data rate mode, data level, and interval in which to make predictions. For the model to perform accurately, `level='sitl'`; however, the SITL-level data is not publicly available and requires an SDC username and password. If you do not have a password for the SDC, you can change to `level='l2'` to use the public database.

The interval can be specified either by orbit number or by `datetime.datetime` object. Normally, predictions are made using the start time of the first sub-region of interest (SROI) and the end time of the last SROI. If the interval is an orbit number, the SROI time intervals are obtained automatically.

In [ ]:
# Data from orbit 1067 SROI1
sc = 'mms1'
level = 'sitl' # 'sitl' (private) or 'l2' (public)
start_interval = 1067
end_interval = 1067
# start_interval = datetime.datetime(2019, 12, 17, 15, 45, 59)
# start_interval = datetime.datetime(2019, 12, 18, 5, 29)

Check the user inputs and define static inputs.

In [ ]:
# SITL data is available in the fast-survey region of the orbit.
# For many instruments, fast- and slow-survey data are combined into a single survey product
mode = 'srvy'

# This script works only for 'sitl' and 'l2' data
if level not in ('sitl', 'l2'):
    raise ValueError('Level must be either "sitl" or "l2".')

# If an orbit number is given, 
if isinstance(start_interval, int):
    sroi = api.mission_events('sroi', start_interval, start_interval, sc=sc)
    start_date = sroi['tstart'][0]
    end_date = sroi['tend'][-1]
else:
    start_date = start_interval
    end_date = end_interval

<a id='setting_up'></a>
## Setting Up
Use the [SDC API](https://lasp.colorado.edu/mms/sdc/public/about/how-to/) to search for available files within the desired time interval.

In [ ]:
# Create an interface to the SDC
mms = api.MrMMS_SDC_API(sc=sc, mode=mode, start_date=start_date, end_date=end_date)

# Ensure that the log-in information is there.
#   - If the config file was already set, this step is redundant.
mms._data_root = pymms.config['data_root']
if mode == 'sitl':
    mms._session.auth(pymms.config['username'], pymms.config['password'])

<a id='download_data'></a>
## Download and Read Data
Start downloading data. We use two helper functions: `read_cdf_vars` to read variable data from CDF files and `quality_factor` to compute a burst trigger quality values as in Section 3.3 of [Phan et al. 2015](http://dx.doi.org/10.1007/s11214-015-0150-2).

The data used here is from the [AFG](http://dx.doi.org/10.1007/s11214-014-0057-3), EDP (from [SDP](http://dx.doi.org/10.1007/s11214-014-0116-9) and [ADP](http://dx.doi.org/10.1007/s11214-014-0115-x)), [DIS, and DES](http://dx.doi.org/10.1007/s11214-016-0245-4) instruments. CDF files are download, data is read and stored as Pandas data frames. Additional metafeatures are calculated.

In [ ]:
def read_cdf_vars(cdf_files, cdf_vars, epoch='Epoch'):
    '''
    Read variables from CDF files into a data frame
    
    Parameters
    ----------
    cdf_files : str or list
        CDF files to be read
    cdf_vars : str or list
        Names of the variables to be read
    epoch : str
        Name of the time variable that serves as the data frame index
    
    Returns
    -------
    out : `pandas.DataFrame`
        The data. If a variable is 2D, "_#" is appended, where "#"
        increases from 0 to var.shape[1]-1.
    '''
    tepoch = epochs.CDFepoch()
    if isinstance(cdf_files, str):
        cdf_files = [cdf_files]
    if isinstance(cdf_vars, str):
        cdf_vars = [cdf_vars]
    if epoch not in cdf_vars:
        cdf_vars.append(epoch)
    
    out = []
    for file in cdf_files:
        file_df = pd.DataFrame()
        cdf = cdfread.CDF(file)
        
        for var_name in cdf_vars:
            # Read the variable data
            data = cdf.varget(var_name)
            if var_name == epoch:
                data = tepoch.to_datetime(data, to_np=True)
            
            # Store as column in data frame
            if data.ndim == 1:
                file_df[var_name] = data
                
            # 2D variables get "_#" appended to name for each column
            elif data.ndim == 2:
                for idx in range(data.shape[1]):
                    file_df['{0}_{1}'.format(var_name, idx)] = data[:,idx]
                    
            # 3D variables gets reshaped to 2D and treated as 2D
            # This includes variables like the pressure and temperature tensors
            elif data.ndim == 3:
                dims = data.shape
                data = data.reshape(dims[0], dims[1]*dims[2])
                for idx in range(data.shape[1]):
                    file_df['{0}_{1}'.format(var_name, idx)] = data[:,idx]
            else:
                print('cdf_var.ndims > 3. Skipping. {0}'.format(var_name))
                continue
        
        # Close the file
        cdf.close()
        
        # Set the epoch variable as the index
        file_df.set_index(epoch, inplace=True)
        out.append(file_df)
    
    # Concatenate all of the file data
    out = pd.concat(out)
    
    # Check that the index is unique
    # Some contiguous low-level data files have data overlap at the edges of the files (e.g., AFG)
    if not out.index.is_unique:
        out['index'] = out.index
        out.drop_duplicates(subset='index', inplace=True, keep='first')
        out.drop(columns='index', inplace=True)
    
    # File names are not always given in order, so sort the data
    out.sort_index(inplace=True)
    return out      

In [ ]:
def rename_df_cols(df, old_col, new_cols):
    '''
    Each column of a multi-dimensional CDF variable gets stored as
    its own independent column in the DataFrame, with "_#" appended
    to the original variable name to indicate which column index
    the column was taken from. This function renames those columns.
    
    Parameters
    ----------
    df : `pandas.DataFrame`
        DataFrame for which the columns are to be renamed
    old_col : str
        Name of the column (sans "_#")
    new_cols : list
        New names to be given to the columns
    '''
    df.rename(columns={'{}_{}'.format(old_col, idx): new_col_name
                       for idx, new_col_name in enumerate(new_cols)},
              inplace=True
             )

In [ ]:
def quality_factor(data, M=2):
    '''
    Compute a quality factor for burst triggers.
    
    Parameters
    ----------
    data : `numpy.ndarray`
        One dimensional data array
    M : int
        Smoothing factor
    
    Returns
    -------
    Q : `numpy.ndarray`
        Burst trigger quality factor
    '''
    smoothed_data = [data[0]]
    for i, value in enumerate(data[1:]):
        smoothed_data.append((smoothed_data[i - 1] * (2 ** M - 1) + value) / 2 ** M)
    return np.subtract(data, smoothed_data)

<a id='fgm_data'></a>
### FGM Data
Download the files

In [ ]:
# There are two magnetometers: AFG and DFG. For L2 data, AFG is
# used for slow survey and DFG is used for fast survey, but are
# known by the instrument name FGM. For SITL-level data, the
# instruments are separate and named as AFG and DFG.
afg_instr = 'afg'
if level == 'l2':
    afg_instr = 'fgm'

afg_mode = mode

# The "SITL"-level data for AFG is labeled "ql" for quick-look
afg_level = level
if level == 'sitl':
    afg_level = 'ql'

afg_optdesc = None

# Download the data files
mms.instr = afg_instr
mms.mode = afg_mode
mms.level = afg_level
mms.optdesc = afg_optdesc
afg_files = mms.download_files()
print(*afg_files, sep='\n')

Read the data

In [ ]:
# Print the variable names from a sample file
afg_cdf = cdfread.CDF(afg_files[0])
info = afg_cdf.cdf_info()
afg_cdf.close()
print(*info['zVariables'], sep='\n')

In [ ]:
# Variable names
t_vname = 'Epoch'
if afg_level == 'l2':
    b_vname = '_'.join((sc, afg_instr, 'b', 'dmpa', afg_mode, afg_level))
else:
    b_vname = '_'.join((sc, afg_instr, afg_mode, 'dmpa'))

# Read the data
afg_df = read_cdf_vars(afg_files, b_vname, epoch=t_vname)

# Rename variables
rename_df_cols(afg_df, b_vname, ('Bx', 'By', 'Bz', '|B|'))

Compute metafeatures and store data in a dataframe.

In [ ]:
# Compute metafeatures
afg_df['P_B'] = afg_df['|B|']**2 / scipy.constants.mu_0
afg_df['clock_angle'] = np.arctan2(afg_df['By'], afg_df['Bz'])
afg_df['Q_dBx'] = quality_factor(afg_df['Bx'])
afg_df.head()

<a id='edp_data'></a>
### EDP Data
Download the files

In [ ]:
edp_instr = 'edp'
edp_optdesc = 'dce'

# EDP does not have "srvy" data, just "fast" and "slow"
edp_mode = mode
if mode == 'srvy':
    edp_mode = 'fast'

# The "SITL"-level data for EDP is labeled "ql" for quick-look
edp_level = level
if level == 'sitl':
    edp_level = 'ql'

# Download the data files
mms.instr = edp_instr
mms.mode = edp_mode
mms.optdesc = edp_optdesc
edp_files = mms.download_files()
print(*edp_files, sep='\n')

Read the files

In [ ]:
# Print the variable names from a sample file
edp_cdf = cdfread.CDF(edp_files[0])
info = edp_cdf.cdf_info()
edp_cdf.close()
print(*info['zVariables'], sep='\n')

In [ ]:
# Variable names
if level == 'l2':
    t_vname = '_'.join((sc, edp_instr, 'epoch', edp_mode, edp_level))
    e_vname = '_'.join((sc, edp_instr, edp_optdesc, 'dsl', edp_mode, edp_level))
else:
    t_vname = '_'.join((sc, edp_instr, edp_optdesc, 'epoch'))
    e_vname = '_'.join((sc, edp_instr, edp_optdesc, 'xyz', 'dsl'))

# Read the data
edp_df = read_cdf_vars(edp_files, e_vname, epoch=t_vname)

# Rename variables
new_vnames = ('Ex', 'Ey', 'Ez')
edp_df.rename(columns={'{}_{}'.format(e_vname, idx): vname
                       for idx, vname in enumerate(new_vnames)},
              inplace=True
             )

Compute metafeatures

In [ ]:
edp_df['|E|'] = np.sqrt(edp_df['Ex']**2 + edp_df['Ey']**2 + edp_df['Ez']**2)
edp_df.head()

<a id='dis_data'></a>
### DIS Data
Download the files

In [ ]:
dis_instr = 'fpi'

# FPI does not have "srvy" data, just "fast" and "slow"
dis_mode = mode
if mode == 'srvy':
    dis_mode = 'fast'

# The "SITL"-level data for FPI is labeled "ql" for quick-look
# There is SITL-level data, but it was discontinued early in the mission
dis_level = level
if level == 'sitl':
    dis_level = 'ql'

dis_optdesc = 'dis'
if level == 'l2':
    dis_optdesc = 'dis-moms'

# Download the data files
mms.instr = dis_instr
mms.mode = dis_mode
mms.level = dis_level
mms.optdesc = dis_optdesc
dis_files = mms.download_files()
print(*dis_files, sep='\n')

Read the files

In [ ]:
# Print the variable names from a sample file
dis_cdf = cdfread.CDF(dis_files[0])
info = dis_cdf.cdf_info()
print(*info['zVariables'], sep='\n')

# Print information about the pressure tensor
# to figure out its dimensions and how the components
# are stored
vname = '_'.join((sc, 'dis', 'prestensor', 'dbcs', dis_mode))
var_notes = dis_cdf.attget(attribute='VAR_NOTES', entry=vname)
print(var_notes['Data'])

# Close the file
dis_cdf.close()

In [ ]:
# Variable names
t_vname = 'Epoch'
espectr_omni_vname = '_'.join((sc, 'dis', 'energyspectr', 'omni', dis_mode))
n_vname = '_'.join((sc, 'dis', 'numberdensity', dis_mode))
v_vname = '_'.join((sc, 'dis', 'bulkv', 'dbcs', dis_mode))
q_heat_vname = '_'.join((sc, 'dis', 'heatq', 'dbcs', dis_mode))
t_para_vname = '_'.join((sc, 'dis', 'temppara', dis_mode))
t_perp_vname = '_'.join((sc, 'dis', 'tempperp', dis_mode))
t_tens_vname = '_'.join((sc, 'dis', 'temptensor', 'dbcs', dis_mode))
p_tens_vname = '_'.join((sc, 'dis', 'prestensor', 'dbcs', dis_mode))

# Read the data
dis_df = read_cdf_vars(dis_files,
                       [espectr_omni_vname, n_vname, v_vname,
                        q_heat_vname, t_para_vname, t_perp_vname,
                        p_tens_vname, t_tens_vname
                       ],
                       epoch=t_vname
                      )

# Rename variables
dis_df.rename(columns={n_vname: 'Ni'}, inplace=True)
dis_df.rename(columns={t_para_vname: 'Ti_para'}, inplace=True)
dis_df.rename(columns={t_perp_vname: 'Ti_perp'}, inplace=True)
rename_df_cols(dis_df, v_vname, ('Vix', 'Viy', 'Viz'))
rename_df_cols(dis_df, q_heat_vname, ('Qi_xx', 'Qi_yy', 'Qi_zz'))
rename_df_cols(dis_df, t_tens_vname,
               ('Ti_xx', 'Ti_xy', 'Ti_xz', 'Ti_yx', 'Ti_yy', 'Ti_yz', 'Ti_zx', 'Ti_zy', 'Ti_zz'))
rename_df_cols(dis_df, p_tens_vname,
               ('Pi_xx', 'Pi_xy', 'Pi_xz', 'Pi_yx', 'Pi_yy', 'Pi_yz', 'Pi_zx', 'Pi_zy', 'Pi_zz'))
rename_df_cols(dis_df, espectr_omni_vname, ['especi_{0}'.format(idx) for idx in range(32)])

# Drop redundant components of the pressure and temperature tensors
dis_df.drop(columns=['Ti_xy', 'Ti_xz', 'Ti_yz', 'Pi_xy', 'Pi_xz', 'Pi_yz'], inplace=True)

Compute metafeatures

In [ ]:
dis_df['Ti_anisotropy'] = (dis_df['Ti_para'] / dis_df['Ti_perp']) - 1
dis_df['Ti_scalar'] = (dis_df['Ti_para'] + 2*dis_df['Ti_perp']) / 3.0
dis_df['Q_dNi'] = quality_factor(dis_df['Ni'])
dis_df['Q_dViz'] = quality_factor(dis_df['Viz'])
Vi_mag = np.sqrt(dis_df['Vix']**2 + dis_df['Viy']**2 + dis_df['Viz']**2)
Pi_ram = dis_df['Ni'] * Vi_mag
dis_df['Q_dPi_ram'] = quality_factor(Pi_ram)

# Drop features that were accidentally excluded
dis_df.drop(columns=['especi_31', 'Viz', 'Qi_zz'], inplace=True)

print(dis_df.columns)
dis_df.head()

<a id='des_data'></a>
### DES Data
Download the files

In [ ]:
des_instr = 'fpi'

# FPI does not have "srvy" data, just "fast" and "slow"
des_mode = mode
if mode == 'srvy':
    des_mode = 'fast'

# The "SITL"-level data for FPI is labeled "ql" for quick-look
# There is SITL-level data, but it was discontinued early in the mission
des_level = level
if level == 'sitl':
    des_level = 'ql'

des_optdesc = 'des'
if level == 'l2':
    des_optdesc = 'des-moms'

# Download the data files
mms.instr = des_instr
mms.mode = des_mode
mms.level = des_level
mms.optdesc = des_optdesc
des_files = mms.download_files()
print(*des_files, sep='\n')

Read the files

In [ ]:
# Print the variable names from a sample file
des_cdf = cdfread.CDF(des_files[0])
info = des_cdf.cdf_info()
print(*info['zVariables'], sep='\n')

# Print information about the pressure tensor
# to figure out its dimensions and how the components
# are stored
vname = 'mms1_des_prestensor_dbcs_fast'
var_notes = des_cdf.attget(attribute='VAR_NOTES', entry=vname)
print(var_notes['Data'])

# Close the file
des_cdf.close()

In [ ]:
# Variable names
t_vname = 'Epoch'
espectr_omni_vname = '_'.join((sc, 'des', 'energyspectr', 'omni', des_mode))
n_vname = '_'.join((sc, 'des', 'numberdensity', des_mode))
v_vname = '_'.join((sc, 'des', 'bulkv', 'dbcs', des_mode))
q_heat_vname = '_'.join((sc, 'des', 'heatq', 'dbcs', des_mode))
t_para_vname = '_'.join((sc, 'des', 'temppara', des_mode))
t_perp_vname = '_'.join((sc, 'des', 'tempperp', des_mode))
t_tens_vname = '_'.join((sc, 'des', 'temptensor', 'dbcs', des_mode))
p_tens_vname = '_'.join((sc, 'des', 'prestensor', 'dbcs', des_mode))

# Read the data
des_df = read_cdf_vars(des_files,
                       [espectr_omni_vname, n_vname, v_vname,
                        q_heat_vname, t_para_vname, t_perp_vname,
                        p_tens_vname, t_tens_vname
                       ],
                       epoch=t_vname
                      )

# Rename variables
des_df.rename(columns={n_vname: 'Ne'}, inplace=True)
des_df.rename(columns={t_para_vname: 'Te_para'}, inplace=True)
des_df.rename(columns={t_perp_vname: 'Te_perp'}, inplace=True)
rename_df_cols(des_df, v_vname, ('Vex', 'Vey', 'Vez'))
rename_df_cols(des_df, q_heat_vname, ('Qe_xx', 'Qe_yy', 'Qe_zz'))
rename_df_cols(des_df, t_tens_vname,
               ('Te_xx', 'Te_xy', 'Te_xz', 'Te_yx', 'Te_yy', 'Te_yz', 'Te_zx', 'Te_zy', 'Te_zz'))
rename_df_cols(des_df, p_tens_vname,
               ('Pe_xx', 'Pe_xy', 'Pe_xz', 'Pe_yx', 'Pe_yy', 'Pe_yz', 'Pe_zx', 'Pe_zy', 'Pe_zz'))
rename_df_cols(des_df, espectr_omni_vname, ['espece_{0}'.format(idx) for idx in range(32)])

# Drop symmetric, redundant components
des_df.drop(columns=['Te_xy', 'Te_xz', 'Te_yz', 'Pe_xy', 'Pe_xz', 'Pe_yz'], inplace=True)

Compute metafeatures

In [ ]:
des_df['Te_anisotropy'] = (des_df['Te_para'] / des_df['Te_perp']) - 1
des_df['Te_scalar'] = (des_df['Te_para'] + 2*des_df['Te_perp']) / 3.0
des_df['Pe_scalar'] = (des_df['Pe_xx'] + des_df['Pe_yy'] + des_df['Pe_zz']) / 3.0
des_df['Q_dNe'] = quality_factor(des_df['Ne'])
des_df['Q_dVez'] = quality_factor(des_df['Vez'])
#Ve_mag = np.sqrt(des_df['Vex']**2 + des_df['Vey']**2 + des_df['Vez']**2)
#Pe_ram = des_df['Ne'] * Ve_mag
#des_df['Q_dPe_ram'] = quality_factor(Pe_ram)


# Drop features that were accidentally excluded
des_df.drop(columns=['espece_31', 'Vez', 'Qe_zz'], inplace=True)

print(des_df.columns)
des_df.head()

<a id='combine_dataframes'></a>
### Combine DataFrames and Compute Additional Metafeatures
Now we will combine all dataframes, downsampling to DES, which as the time index with the longest sampling period (`4.5s`). After that, multi-instrument metafeatures are calculated.

In [ ]:
# Resample data
afg_df = afg_df.reindex(des_df.index, method='nearest')
edp_df = edp_df.reindex(des_df.index, method='nearest')
dis_df = dis_df.reindex(des_df.index, method='nearest')

# Merge dataframes
df = des_df
df = df.join(dis_df, how='outer')
df = df.join(afg_df, how='outer')
df = df.join(edp_df, how='outer')

# Metafeatures
df['T_ratio'] = df['Ti_scalar'] / df['Ti_scalar']
df['plasma_beta'] = (df['Ti_scalar'] + df['Ti_scalar']) / df['|E|']

In [ ]:
print(*df.columns, sep=', ')
df.head()

<a id='run_model'></a>
## Run the Model

In [ ]:
def roundTime(dt=None, dateDelta=datetime.timedelta(minutes=1)):
    """Round a datetime object to a multiple of a timedelta
    
    Parameters
    ----------
    dt : `datetime.datetime` = datetime.datetime.now().
    dateDelta : `datetime.timedelta`
        we round to a multiple of this, default 1 minute.
    
    Author: Thierry Husson 2012 - Use it as you want but don't blame me.
            Stijn Nevens 2014 - Changed to use only datetime objects as variables
    """
    roundTo = dateDelta.total_seconds()

    if dt == None : dt = datetime.datetime.now()
    seconds = (dt - dt.min).seconds
    # // is a floor division, not a comment on following line:
    rounding = (seconds+roundTo/2) // roundTo * roundTo
    return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)

In [ ]:
def lstm(num_features=123, layer_size=300):
    """
    Helper function to define the LSTM used to make predictions.
    """
    model = Sequential()

    model.add(Bidirectional(LSTM(layer_size, return_sequences=True, activation='tanh', recurrent_activation='sigmoid'),
                            input_shape=(None, num_features)))

    model.add(Dropout(0.4))

    model.add(Bidirectional(LSTM(layer_size, return_sequences=True, activation='tanh', recurrent_activation='sigmoid')))

    model.add(Dropout(0.4))

    model.add(TimeDistributed(Dense(1, activation='sigmoid')))

    return model

In [ ]:
# # Define MMS CDF directory location
# Load model
model = lstm()
model.load_weights(str(model_root / 'model_weights.h5'))

# Interpolate interior values, drop outside rows containing 0s
data = df.replace([np.inf, -np.inf], np.nan)
data = data.interpolate(method='time', limit_area='inside')
data = data.loc[(df != 0).any(axis=1)]

# Select data within time range
data = data.loc[start_date:end_date]
data_index = data.index

# Scale data
scaler = pickle.load(open(model_root / 'scaler.sav', 'rb'))
data = scaler.transform(data)

# Run data through model
predictions_list = model.predict(np.expand_dims(data, axis=0))

# Filter predictions with threshold
threshold = 0.5
filtered_output = [0 if x < threshold else 1 for x in predictions_list.squeeze()]

# Create selections from predictions
predictions_df = pd.DataFrame()
predictions_df.insert(0, "time", data_index)
predictions_df.insert(1, "prediction", filtered_output)
predictions_df['group'] = (predictions_df.prediction != predictions_df.prediction.shift()).cumsum()
predictions_df = predictions_df.loc[predictions_df['prediction'] == 1]
selections = pd.DataFrame({'BeginDate': predictions_df.groupby('group').time.first().map(lambda x: roundTime(utc2tai(x), datetime.timedelta(seconds=10))),
                           'EndDate': predictions_df.groupby('group').time.last().map(lambda x: roundTime(utc2tai(x), datetime.timedelta(seconds=10)))})
selections = selections.set_index('BeginDate')

selections['score'] = "150.0" # This is a placeholder for the FOM
selections['description'] = "MP crossing (automatically generated)"

# Create the file name: gls_selections_<model-name>_<current-time-as-YYYY-MM-DD-HH-MM-SS>.csv
current_datetime = datetime.datetime.now()
selections_filetime = current_datetime.strftime('%Y-%m-%d-%H-%M-%S')
file_name = 'gls_selections_mp-dl-unh_{0}.csv'.format(selections_filetime)

# Output selections
print('Saving selections to CSV: {0}'.format(file_name))
if not dropbox_root.exists():
    dropbox_root.mkdir(parents=True)
selections.to_csv(dropbox_root / file_name, header=False)